In [1]:
# Import packages
import ee
import geemap
import os
import geopandas as gpd
import folium

In [ ]:
#image.sampleRectangle(region=geom, )

In [2]:
# Create a map
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [3]:
# Add an image to the map
dataset = ee.Image('USGS/GFSAD1000_V1') # import the total image
landcover = ee.Image(dataset.select('landcover')) # setect the band we want to add
Map.addLayer(landcover, {}, 'GFSAD') # add this band as a layer to the map

In [ ]:
# NOTE: THIS FEATURE WILL NOT BE USED ANYMORE SINCE WE HAVE A POLYGON OF TELANGANA GIVEN
# Draw any shapes on the map using the Drawing tools before executing this code block
# feature = Map.draw_last_feature # region of interest
# feature
# roi = feature.geometry() 

In [5]:
# Import this file to derive the polygon geometry for Telangana
bound_Telangana = gpd.read_file('C:/Users/mieke/Documents/Msc Thesis/GitHub repositories/dicra/analytics/geospatial_internship/datasets/telangana_shapefile.geojson')
bound_Telangana.head()

,GID_0,NAME_0,GID_1,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,geometry
0,IND,India,IND.32_1,Telangana,None,None,State,State,None,IN.TG,"MULTIPOLYGON (((79.21147 16.36281, 79.22324 16..."


In [6]:
# Extract longitudes and latitudes from the Multipolygon geometry of Telangana
lon, lat = bound_Telangana.geometry[0][0].exterior.coords.xy
# Create a list containing all pairs of longitudes and latitudes of Telangana
list_of_coordinates = []
for a,b in zip(lon, lat):
    list_of_coordinates += [(a,b)]
# Create a Polygon geometry of Telangana
roi = ee.Geometry.Polygon(list_of_coordinates)

C:\Users\mieke\AppData\Local\Temp\ipykernel_7244\2973172409.py:2: ShapelyDeprecationWarning: __getitem__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  lon, lat = bound_Telangana.geometry[0][0].exterior.coords.xy


In [7]:
image_crop = landcover.clip(roi) # Crop the image(band) to the area of the derived Polygon 
print(type(image_crop)) # ee.image.Image

<class 'ee.image.Image'>


In [8]:
# Export this image to a geotif file
geemap.ee_export_image(image_crop, filename='C:/Users/mieke/Documents/Msc Thesis/Notebooks Python/GFSAD/GFSAD_Telangana.tif', scale=100, region=roi, file_per_band=True)
# If you get a size error, then change the scale

Generating URL ...
Please wait ...
Data downloaded to C:\Users\mieke\Documents\Notebooks Python\GFSAD


In [9]:
# Add layers to a folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = 'Map Data &copy; <a href "https://earthengine.google.com/"> Google Earth Engine </a>',
        name = name,
        overlay = True,
        control = True,
        opacity=1
    ).add_to(self)
# Add this function to the Folium Map object.
folium.Map.add_ee_layer = add_ee_layer

In [10]:
map_18 = folium.Map(location = [17,80], zoom_start = 6) # Create a map
# State the visualizations of the new layer
visualization = {'min': 0, 'max': 5, 'bands': ['landcover'], 'palette': ['black', 'orange', 'brown', '02a50f', 'green', 'yellow']}
map_18.add_ee_layer(image_crop, vis_params = visualization, name = 'GFSAD1000') # add the new layer to the map
map_18 # Plot the map